In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F 
import torch.utils.data
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [ ]:
#load saved tensors
#these are from the training set
train_loader = torch.load('train_tensor.pt')
test_loader = torch.load('test_tensor.pt')

data_folder = './MNIST/data/'
kaggle_filename = data_folder + 'test_images.pkl'
kaggle_images = pd.read_pickle(kaggle_filename)

torch_kaggle = torch.from_numpy(kaggle_images).type(torch.LongTensor)
torch_kaggle = torch_kaggle.view(-1, 1,64,64).float()

kaggle = torch.utils.data.TensorDataset(torch_kaggle)
kaggle_loader = torch.utils.data.DataLoader(kaggle, batch_size = 1, shuffle = False)

In [ ]:
#add your model here
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), #in, out
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #we now have 32*32 images
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #now its 16*16
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #now its 8*8
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(8 * 8 * 128, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
#initialize your model
model = ConvNet()

In [ ]:
#load your models state
model.load_state_dict(torch.load('/home/botond/Boti/uni/mcgill/comp551/a3/deep_cnn_model.ckpt'))

In [ ]:
#predict
predictions = []
model.eval()
for x in kaggle_loader:
    outputs = model(x[0])
    predicted = torch.max(outputs.data, 1)
    #print(predicted[1].item())
    predictions.append(predicted[1].item())
    print(predictions)

In [ ]:
print(len(predictions))

In [ ]:
x = [x for x in range(0,10000)]
df = pd.DataFrame(data={"ID": x, "Category": predictions})
df.to_csv("./submission.csv", sep=',',index=False)